In [82]:
import os, re, json, math
from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)

RNG_SEED = 42
np.random.seed(RNG_SEED)

PROVENANCE = {"steps": []}
def log_step(title, details=None):
    PROVENANCE["steps"].append({"title": title, "details": details or {}})

def report(df, name="df"):
    print(f"--- {name} ---")
    print(f"shape: {df.shape}")
    print("\n.dtypes:\n", df.dtypes)
    print("\n# missing by column:\n", df.isna().sum().sort_values(ascending=False).head(20))
    print("\n.head():")
    display(df.head(5))
    return df.shape

def missing_table(df):
    m = df.isna().sum()
    pct = (m / len(df) * 100).round(2)
    return pd.DataFrame({"missing": m, "pct": pct}).sort_values("pct", ascending=False)

def to_numeric_smart(series):
    return (series.astype(str)
                 .str.replace(r"[,$ ]", "", regex=True)
                 .str.replace(r"[kK]$", "000", regex=True)
                 .str.replace(r"[^\d\.\-]", "", regex=True)
                 .replace({"": np.nan})
                 .astype(float))

def to_snake(name: str) -> str:
    name = re.sub(r"[\s/]+", "_", name.strip())
    name = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", name)
    return name.lower()

In [ ]:
#cell 1 — Load raw files
DATA_DIR = Path("./data")
EMP_FEATURES_PATH = DATA_DIR / "Employee_dataset.csv"
EMP_SALARY_PATH   = DATA_DIR / "Employee_salaries.csv"

assert EMP_FEATURES_PATH.exists(), f"Missing: {EMP_FEATURES_PATH.resolve()}"
assert EMP_SALARY_PATH.exists(),   f"Missing: {EMP_SALARY_PATH.resolve()}"

print("FEATURES file:", EMP_FEATURES_PATH.resolve())
print("SALARY   file:", EMP_SALARY_PATH.resolve())

read_kwargs = dict(low_memory=False)
df_feat = pd.read_csv(EMP_FEATURES_PATH, **read_kwargs)
df_sal  = pd.read_csv(EMP_SALARY_PATH, **read_kwargs)

report(df_feat, "df_feat (raw)")
report(df_sal,  "df_sal (raw)")
log_step("load_raw", {"features_path": str(EMP_FEATURES_PATH), "salary_path": str(EMP_SALARY_PATH)})

FEATURES file: /Users/yongryan/Downloads/bigdataryan /data/Employee_dataset.csv
SALARY   file: /Users/yongryan/Downloads/bigdataryan /data/Employee_salaries.csv
--- df_feat (raw) ---
shape: (1000000, 8)

.dtypes:
 jobId               object
companyId           object
jobRole             object
education           object
major               object
industry            object
yearsExperience    float64
distanceFromCBD    float64
dtype: object

# missing by column:
 industry           214
major              207
yearsExperience    198
education          186
distanceFromCBD    166
jobRole            165
companyId          148
jobId              105
dtype: int64

.head():


,jobId,companyId,jobRole,education,major,industry,yearsExperience,distanceFromCBD
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10.0,83.0
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3.0,73.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


--- df_sal (raw) ---
shape: (1000000, 2)

.dtypes:
 jobId                 object
salaryInThousands    float64
dtype: object

# missing by column:
 salaryInThousands    229
jobId                223
dtype: int64

.head():


,jobId,salaryInThousands
0,JOB1362684407687,130.0
1,JOB1362684407688,101.0
2,JOB1362684407689,137.0
3,JOB1362684407690,142.0
4,JOB1362684407691,163.0


In [ ]:
#cell 2 — Standardize column names & trim strings (safe NA), normalize job_id
df_feat.columns = [to_snake(c) for c in df_feat.columns]
df_sal.columns  = [to_snake(c)  for c in df_sal.columns]

# preserve genuine missing with pandas' StringDtype 
for c in df_feat.select_dtypes(include="object").columns:
    df_feat[c] = pd.Series(df_feat[c], dtype="string").str.strip()
for c in df_sal.select_dtypes(include="object").columns:
    df_sal[c] = pd.Series(df_sal[c], dtype="string").str.strip()

#normalize job_id; keep only valid IDs; keep <NA> as missing
def normalize_job_id(s: pd.Series) -> pd.Series:
    s = pd.Series(s, dtype="string").str.strip().str.upper()
    s = s.replace(r"^\s*$", pd.NA, regex=True)
    ok = s.str.fullmatch(r"JOB\d+")
    return s.where(ok, pd.NA)

assert "job_id" in df_feat.columns, "job_id not found in features"
assert "job_id" in df_sal.columns,  "job_id not found in salary"
df_feat["job_id"] = normalize_job_id(df_feat["job_id"])
df_sal["job_id"]  = normalize_job_id(df_sal["job_id"])

report(df_feat, "df_feat (standardized/trimmed)")
report(df_sal,  "df_sal (standardized/trimmed)")
log_step("standardize_columns")


--- df_feat (standardized/trimmed) ---
shape: (1000000, 8)

.dtypes:
 job_id               string[python]
company_id           string[python]
job_role             string[python]
education            string[python]
major                string[python]
industry             string[python]
years_experience            float64
distance_from_cbd           float64
dtype: object

# missing by column:
 industry             214
major                207
years_experience     198
education            186
distance_from_cbd    166
job_role             165
company_id           148
job_id               105
dtype: int64

.head():


,job_id,company_id,job_role,education,major,industry,years_experience,distance_from_cbd
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10.0,83.0
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3.0,73.0
2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN


--- df_sal (standardized/trimmed) ---
shape: (1000000, 2)

.dtypes:
 job_id                 string[python]
salary_in_thousands           float64
dtype: object

# missing by column:
 salary_in_thousands    229
job_id                 223
dtype: int64

.head():


,job_id,salary_in_thousands
0,JOB1362684407687,130.0
1,JOB1362684407688,101.0
2,JOB1362684407689,137.0
3,JOB1362684407690,142.0
4,JOB1362684407691,163.0


In [ ]:
#cell 3 — type coercions for known numerics (no stringifying NA)
def to_numeric_smart_safe(series: pd.Series) -> pd.Series:
    s = pd.Series(series, dtype="string")
    s = s.str.replace(r"[,$ ]", "", regex=True)
    s = s.str.replace(r"[kK]$", "000", regex=True)
    s = s.str.replace(r"[^\d\.\-]", "", regex=True)
    s = s.replace({"": pd.NA})
    return pd.to_numeric(s, errors="coerce")

num_feat_cols = [c for c in ["years_experience", "distance_from_cbd"] if c in df_feat.columns]
for col in num_feat_cols:
    df_feat[col] = to_numeric_smart_safe(df_feat[col])

if "salary_in_thousands" in df_sal.columns:
    df_sal["salary_in_thousands"] = to_numeric_smart_safe(df_sal["salary_in_thousands"])

report(df_feat[num_feat_cols] if num_feat_cols else df_feat, "df_feat (typed)")
report(df_sal, "df_sal (typed)")
log_step("type_coercions", {"feature_numerics": num_feat_cols})


--- df_feat (typed) ---
shape: (1000000, 2)

.dtypes:
 years_experience     Float64
distance_from_cbd    Float64
dtype: object

# missing by column:
 years_experience     198
distance_from_cbd    166
dtype: int64

.head():


,years_experience,distance_from_cbd
0,10.0,83.0
1,3.0,73.0
2,<NA>,<NA>
3,<NA>,<NA>
4,<NA>,<NA>


--- df_sal (typed) ---
shape: (1000000, 2)

.dtypes:
 job_id                 string[python]
salary_in_thousands           Float64
dtype: object

# missing by column:
 salary_in_thousands    229
job_id                 223
dtype: int64

.head():


,job_id,salary_in_thousands
0,JOB1362684407687,130.0
1,JOB1362684407688,101.0
2,JOB1362684407689,137.0
3,JOB1362684407690,142.0
4,JOB1362684407691,163.0


In [ ]:
#cell 4 — drop null/invalid job_id rows & de-duplicate by job_id (keep first)
drop_feat_null = int(df_feat["job_id"].isna().sum())
drop_sal_null  = int(df_sal["job_id"].isna().sum())

df_feat = df_feat[df_feat["job_id"].notna()].copy()
df_sal  = df_sal[df_sal["job_id"].notna()].copy()

before_feat = len(df_feat); before_sal = len(df_sal)
df_feat = df_feat.drop_duplicates("job_id", keep="first")
df_sal  = df_sal.drop_duplicates("job_id", keep="first")

print(f"df_feat dedup: {before_feat:,} -> {len(df_feat):,}")
print(f"df_sal  dedup: {before_sal:,} -> {len(df_sal):,}")

log_step("deduplicate", {
    "feat_null_job_id_dropped": drop_feat_null,
    "sal_null_job_id_dropped":  drop_sal_null,
    "feat_removed_dups": int(before_feat - len(df_feat)),
    "sal_removed_dups":  int(before_sal  - len(df_sal)),
})


df_feat dedup: 999,895 -> 999,895
df_sal  dedup: 999,777 -> 999,777


In [ ]:
#cell 4c — cross-dataset linkage on distinct job_ids
feat_keys = df_feat[["job_id"]].drop_duplicates()
sal_keys  = df_sal[["job_id"]].drop_duplicates()

inter_n     = feat_keys.merge(sal_keys, on="job_id", how="inner").shape[0]
only_feat_n = len(feat_keys) - inter_n
only_sal_n  = len(sal_keys)  - inter_n

print(f"Distinct job_ids — features: {len(feat_keys):,} | salary: {len(sal_keys):,} | intersection: {inter_n:,}")
print(f"Only in features: {only_feat_n:,} | Only in salary: {only_sal_n:,}")

#show tiny samples if mismatches remain
if only_feat_n:
    sample_only_feat = (feat_keys.merge(sal_keys, on="job_id", how="left", indicator=True)
                        .query("_merge=='left_only'").head(5))
    print("\nSample job_ids only in features:"); display(sample_only_feat)
if only_sal_n:
    sample_only_sal  = (sal_keys.merge(feat_keys, on="job_id", how="left", indicator=True)
                        .query("_merge=='left_only'").head(5))
    print("\nSample job_ids only in salary:"); display(sample_only_sal)

log_step("key_linkage_fast", {
    "feat_distinct": int(len(feat_keys)),
    "sal_distinct":  int(len(sal_keys)),
    "intersection":  int(inter_n),
    "only_feat":     int(only_feat_n),
    "only_sal":      int(only_sal_n),
})

Distinct job_ids — features: 999,895 | salary: 999,777 | intersection: 999,676
Only in features: 219 | Only in salary: 101

Sample job_ids only in features:


,job_id,_merge
184,JOB1362684407883,left_only
185,JOB1362684407884,left_only
186,JOB1362684407885,left_only
187,JOB1362684407886,left_only
188,JOB1362684407887,left_only



Sample job_ids only in salary:


,job_id,_merge
2,JOB1362684407689,left_only
3,JOB1362684407690,left_only
4,JOB1362684407691,left_only
5,JOB1362684407692,left_only
49,JOB1362684407740,left_only


In [ ]:
#cell 5— canonicalize categoricals (consistent vocab + dtype)

CANON = {
    "education": {
        # map lowercase cleaned tokens to canonical
        "none": "NONE",
        "high_school": "HIGH_SCHOOL",
        "hs": "HIGH_SCHOOL",
        "bachelor": "BACHELOR",
        "bachelors": "BACHELOR",
        "master": "MASTER",
        "masters": "MASTER",
        "phd": "PHD",
        "doctorate": "PHD",
    },
    "industry": {
        "health": "HEALTH",
        "healthcare": "HEALTH",
        "web": "WEB",
        "tech": "WEB",
        "finance": "FINANCE",
        "financial": "FINANCE",
        "service": "SERVICE",
        "services": "SERVICE",
        "education": "EDUCATION",
        "auto": "AUTO",
        "oil": "OIL",
    },
    "major": {
        "none": "NONE",
        "na": "NONE",
        "math": "MATH",
        "physics": "PHYSICS",
        "cs": "CS",
        "computer_science": "CS",
        "business": "BUSINESS",
        "econ": "ECON",
        "economics": "ECON",
    },
    "job_role": {
        "ceo": "CEO",
        "cfo": "CFO",
        "junior": "JUNIOR",
        "janitor": "JANITOR",
        "vice_president": "VICE_PRESIDENT",
        "vp": "VICE_PRESIDENT",
        "manager": "MANAGER",
        "engineer": "ENGINEER",
        "analyst": "ANALYST",
    },
}

def normalize_token(x: pd.Series) -> pd.Series:
    # uniform formatting then lookup in each column-specific map
    s = pd.Series(x, dtype="string[python]").str.strip().str.replace(r"\s+", "_", regex=True).str.lower()
    return s

def canon_map(series: pd.Series, mapping: dict) -> pd.Series:
    s = normalize_token(series)
    s = s.map(mapping).fillna(s.str.upper())  # default to upper for unknowns
    return s.astype("string[python]")

cat_cols = [c for c in ["education", "industry", "major", "job_role"] if c in df_feat.columns]
if "education" in cat_cols:
    df_feat["education"] = canon_map(df_feat["education"], CANON["education"])
if "industry" in cat_cols:
    df_feat["industry"]  = canon_map(df_feat["industry"],  CANON["industry"])
if "major" in cat_cols:
    df_feat["major"]     = canon_map(df_feat["major"],     CANON["major"])
if "job_role" in cat_cols:
    df_feat["job_role"]  = canon_map(df_feat["job_role"],  CANON["job_role"])

# canonicalize 'company_id' shape (UPPER) while preserving NA
if "company_id" in df_feat.columns:
    df_feat["company_id"] = pd.Series(df_feat["company_id"], dtype="string[python]").str.upper()

report(df_feat[cat_cols], "df_feat (categoricals canon)")
log_step("categorical_canonicalization", {"cols": cat_cols})


--- df_feat (categoricals canon) ---
shape: (999895, 4)

.dtypes:
 education    string[python]
industry     string[python]
major        string[python]
job_role     string[python]
dtype: object

# missing by column:
 industry     144
major        131
education    105
job_role      81
dtype: int64

.head():


,education,industry,major,job_role
0,MASTER,HEALTH,MATH,CFO
1,HIGH_SCHOOL,WEB,NONE,CEO
10,HIGH_SCHOOL,HEALTH,NONE,JANITOR
11,MASTER,EDUCATION,PHYSICS,CEO
12,NONE,OIL,NONE,JUNIOR


In [ ]:
#cell6 — missingness: minimal impute + dtype restore + COMP_UNKNOWN sentinel for company_id

# numeric: median imputation (only these columns if present)
num_cols = [c for c in ["years_experience", "distance_from_cbd"] if c in df_feat.columns]
for c in num_cols:
    if df_feat[c].isna().any():
        df_feat[c] = df_feat[c].fillna(df_feat[c].median())

# categorical (feature) imputation via mode (exclude company_id here)
cat_cols = [c for c in ["job_role", "education", "major", "industry"] if c in df_feat.columns]
for c in cat_cols:
    if df_feat[c].isna().any():
        mode_val = df_feat[c].mode(dropna=True)
        if len(mode_val):
            df_feat[c] = df_feat[c].fillna(mode_val.iloc[0])

#sentinel for company_id: fill NA as 'COMP_UNKNOWN' for EDA convenience 
if "company_id" in df_feat.columns:
    # ensure string dtype first
    df_feat["company_id"] = pd.Series(df_feat["company_id"], dtype="string[python]")
    missing_companies = int(df_feat["company_id"].isna().sum())
    if missing_companies > 0:
        df_feat["company_id"] = df_feat["company_id"].fillna("COMP_UNKNOWN")
        log_step("company_id_sentinel", {"filled": missing_companies, "value": "COMP_UNKNOWN"})

# restore StringDtype for all categorical/string cols (including company_id)
dtype_string_cols = [c for c in ["company_id", "job_role", "education", "major", "industry"] if c in df_feat.columns]
for c in dtype_string_cols:
    df_feat[c] = pd.Series(df_feat[c], dtype="string[python]")

report(df_feat[num_cols + dtype_string_cols], "df_feat (imputed + dtypes restored + company_id sentinel)")
log_step("impute_and_restore_dtypes", {
    "numeric_imputed": num_cols,
    "categorical_mode_imputed": cat_cols,
    "company_id_sentinel": True if "company_id" in df_feat.columns else False
})


--- df_feat (imputed + dtypes restored + company_id sentinel) ---
shape: (999895, 7)

.dtypes:
 years_experience            Float64
distance_from_cbd           Float64
company_id           string[python]
job_role             string[python]
education            string[python]
major                string[python]
industry             string[python]
dtype: object

# missing by column:
 years_experience     0
distance_from_cbd    0
company_id           0
job_role             0
education            0
major                0
industry             0
dtype: int64

.head():


,years_experience,distance_from_cbd,company_id,job_role,education,major,industry
0,10.0,83.0,COMP37,CFO,MASTER,MATH,HEALTH
1,3.0,73.0,COMP19,CEO,HIGH_SCHOOL,NONE,WEB
10,24.0,30.0,COMP56,JANITOR,HIGH_SCHOOL,NONE,HEALTH
11,7.0,79.0,COMP7,CEO,MASTER,PHYSICS,EDUCATION
12,8.0,29.0,COMP4,JUNIOR,NONE,NONE,OIL


In [ ]:
# cell 7 — sanity constraints (non-negatives, practical caps), then re-impute numerics if we created NA
import pandas as pd
import numpy as np

issues = {}

def clamp_nonnegative(df, col):
    """Set negatives to NA (nullable Float64) and return how many were changed."""
    neg_count = int((df[col] < 0).sum())
    if neg_count > 0:
        df.loc[df[col] < 0, col] = pd.NA
    return neg_count

# 1) enforce non-negatives on numeric features
for col in ["years_experience", "distance_from_cbd"]:
    if col in df_feat.columns:
        negs = clamp_nonnegative(df_feat, col)
        issues[col] = {"negatives_found": negs}

# 2) cap years_experience to a practical upper bound (retain information, avoid absurd values)
if "years_experience" in df_feat.columns:
    too_high = int((df_feat["years_experience"] > 50).sum())
    df_feat.loc[df_feat["years_experience"] > 50, "years_experience"] = 50
    issues.setdefault("years_experience", {}).update({">50_capped": too_high})

# 3) handle outliers for distance_from_cbd by capping to a plausible upper bound (e.g., city radius 150)
if "distance_from_cbd" in df_feat.columns:
    # snapshot examples before capping for provenance/audit
    before_cap = df_feat.loc[df_feat["distance_from_cbd"] > 150, ["job_id", "distance_from_cbd"]].copy()
    # clamp to [0, 150]
    df_feat["distance_from_cbd"] = df_feat["distance_from_cbd"].clip(lower=0, upper=150)
    rows_capped = int(len(before_cap))
    if rows_capped > 0:
        log_step("distance_cap", {
            "cap_upper": 150,
            "rows_capped": rows_capped,
            "examples": before_cap.head(5).to_dict(orient="records")
        })
    issues.setdefault("distance_from_cbd", {}).update({"capped_above_150": rows_capped})

# 4) re-impute numerics if new NA were introduced by clamps (keeps feature NA = 0 post-cleaning)
for col in ["years_experience", "distance_from_cbd"]:
    if col in df_feat.columns and df_feat[col].isna().any():
        med = float(df_feat[col].median())
        fill_n = int(df_feat[col].isna().sum())
        df_feat[col] = df_feat[col].fillna(med)
        issues.setdefault(col, {}).update({"post_sanity_impute_filled": fill_n, "median": med})

report(df_feat, "df_feat (sanity-checked)")
log_step("sanity_constraints", issues)


--- df_feat (sanity-checked) ---
shape: (999895, 8)

.dtypes:
 job_id               string[python]
company_id           string[python]
job_role             string[python]
education            string[python]
major                string[python]
industry             string[python]
years_experience            Float64
distance_from_cbd           Float64
dtype: object

# missing by column:
 job_id               0
company_id           0
job_role             0
education            0
major                0
industry             0
years_experience     0
distance_from_cbd    0
dtype: int64

.head():


,job_id,company_id,job_role,education,major,industry,years_experience,distance_from_cbd
0,JOB1362684407687,COMP37,CFO,MASTER,MATH,HEALTH,10.0,83.0
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3.0,73.0
10,JOB1362684407697,COMP56,JANITOR,HIGH_SCHOOL,NONE,HEALTH,24.0,30.0
11,JOB1362684407698,COMP7,CEO,MASTER,PHYSICS,EDUCATION,7.0,79.0
12,JOB1362684407699,COMP4,JUNIOR,NONE,NONE,OIL,8.0,29.0


In [ ]:
#  cell 8 — Outlier flags (IQR) for salary & key numerics (no target capping)

def iqr_bounds(series: pd.Series):
    """Return Tukey IQR lower/upper bounds. Handles small/empty series safely."""
    s = series.dropna()
    if len(s) < 2:
        return (np.nan, np.nan)
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    return float(q1 - 1.5 * iqr), float(q3 + 1.5 * iqr)

outlier_info = {}

#target (salary) — FLAG ONLY (do not cap/drop here) 
if "salary_in_thousands" in df_sal.columns:
    lo, hi = iqr_bounds(df_sal["salary_in_thousands"])
    if np.isfinite(lo) and np.isfinite(hi):
        df_sal["is_outlier_salary"] = (
            (df_sal["salary_in_thousands"] < lo) | (df_sal["salary_in_thousands"] > hi)
        ).astype("boolean")
        outlier_info["salary_in_thousands"] = {
            "lo": lo, "hi": hi,
            "count_true": int(df_sal["is_outlier_salary"].sum(skipna=True)),
            "pct_true": float((df_sal["is_outlier_salary"].sum(skipna=True) / len(df_sal)) * 100.0),
        }
    else:
        # if not enough data to compute bounds, create a nullable boolean column of all <NA>
        df_sal["is_outlier_salary"] = pd.Series(pd.array([pd.NA] * len(df_sal), dtype="boolean"))
        outlier_info["salary_in_thousands"] = {"lo": lo, "hi": hi, "count_true": 0, "pct_true": 0.0}

#feature columns — optional flags (OK to cap features in Cell 7, but here we only flag)
for col in ["years_experience", "distance_from_cbd"]:
    if col in df_feat.columns:
        lo, hi = iqr_bounds(df_feat[col])
        if np.isfinite(lo) and np.isfinite(hi):
            flag_col = f"is_outlier_{col}"
            df_feat[flag_col] = ((df_feat[col] < lo) | (df_feat[col] > hi)).astype("boolean")
            outlier_info[col] = {
                "lo": lo, "hi": hi,
                "count_true": int(df_feat[flag_col].sum(skipna=True)),
                "pct_true": float((df_feat[flag_col].sum(skipna=True) / len(df_feat)) * 100.0),
            }
        else:
            flag_col = f"is_outlier_{col}"
            df_feat[flag_col] = pd.Series(pd.array([pd.NA] * len(df_feat), dtype="boolean"))
            outlier_info[col] = {"lo": lo, "hi": hi, "count_true": 0, "pct_true": 0.0}

#summary
print("Outlier thresholds & counts:")
print(json.dumps(outlier_info, indent=2))

#quick peeks
if "is_outlier_salary" in df_sal.columns:
    display(df_sal.loc[df_sal["is_outlier_salary"] == True, ["job_id", "salary_in_thousands"]].head(5))
for col in ["years_experience", "distance_from_cbd"]:
    f = f"is_outlier_{col}"
    if f in df_feat.columns:
        display(df_feat.loc[df_feat[f] == True, ["job_id", col]].head(5))

log_step("outlier_flags", outlier_info)


Outlier thresholds & counts:
{
  "salary_in_thousands": {
    "lo": 8.5,
    "hi": 220.5,
    "count_true": 7120,
    "pct_true": 0.7121588114149455
  },
  "years_experience": {
    "lo": -12.0,
    "hi": 36.0,
    "count_true": 0,
    "pct_true": 0.0
  },
  "distance_from_cbd": {
    "lo": -50.0,
    "hi": 150.0,
    "count_true": 0,
    "pct_true": 0.0
  }
}


,job_id,salary_in_thousands
266,JOB1362684407953,223.0
362,JOB1362684408049,223.0
560,JOB1362684408247,248.0
670,JOB1362684408357,240.0
719,JOB1362684408406,225.0


,job_id,years_experience


,job_id,distance_from_cbd


In [ ]:
#cell 9 — join features ↔ salary (left join; one-to-one after dedup)
cols_from_sal = ["job_id", "salary_in_thousands"]
if "is_outlier_salary" in df_sal.columns:
    cols_from_sal.append("is_outlier_salary")

df_full = df_feat.merge(df_sal[cols_from_sal], on="job_id", how="left", validate="one_to_one")

#ensure nullable boolean remains boolean after merge
if "is_outlier_salary" in df_full.columns:
    df_full["is_outlier_salary"] = df_full["is_outlier_salary"].astype("boolean")

report(df_full, "df_full (post-join)")
log_step("join_merge", {"how": "left", "rows": len(df_full)})


--- df_full (post-join) ---
shape: (999895, 12)

.dtypes:
 job_id                          string[python]
company_id                      string[python]
job_role                        string[python]
education                       string[python]
major                           string[python]
industry                        string[python]
years_experience                       Float64
distance_from_cbd                      Float64
is_outlier_years_experience            boolean
is_outlier_distance_from_cbd           boolean
salary_in_thousands                    Float64
is_outlier_salary                      boolean
dtype: object

# missing by column:
 is_outlier_salary               225
salary_in_thousands             225
job_id                            0
company_id                        0
education                         0
job_role                          0
major                             0
industry                          0
distance_from_cbd                 0
years_experience

,job_id,company_id,job_role,education,major,industry,years_experience,distance_from_cbd,is_outlier_years_experience,is_outlier_distance_from_cbd,salary_in_thousands,is_outlier_salary
0,JOB1362684407687,COMP37,CFO,MASTER,MATH,HEALTH,10.0,83.0,False,False,130.0,False
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3.0,73.0,False,False,101.0,False
2,JOB1362684407697,COMP56,JANITOR,HIGH_SCHOOL,NONE,HEALTH,24.0,30.0,False,False,102.0,False
3,JOB1362684407698,COMP7,CEO,MASTER,PHYSICS,EDUCATION,7.0,79.0,False,False,144.0,False
4,JOB1362684407699,COMP4,JUNIOR,NONE,NONE,OIL,8.0,29.0,False,False,79.0,False


In [ ]:
#cell 10 — final validations & save cleaned outputs
from pathlib import Path
import json

# 1) invariants on keys and join
assert "job_id" in df_full.columns, "job_id required"
assert len(df_full) == len(df_feat), "Left join invariant broken (row loss)"
assert df_full["job_id"].isna().sum() == 0, "job_id must be non-null"
assert df_full["job_id"].str.fullmatch(r"JOB\d+").all(), "invalid job_id pattern present"
assert df_full["job_id"].is_unique, "job_id must be unique after dedupe"

# 2) structural consistency for company_id:
#  allow either pattern COMP\d+ OR the sentinel COMP_UNKNOWN
if "company_id" in df_full.columns:
    cid = pd.Series(df_full["company_id"], dtype="string[python]")
    valid_pat = cid.str.fullmatch(r"COMP\d+")
    is_sentinel = cid.eq("COMP_UNKNOWN")
    assert (valid_pat | is_sentinel).fillna(False).all(), "invalid company_id value (must be COMP#### or COMP_UNKNOWN)"

# 3) persist artifacts
SAVE_DIR = Path("./artifacts/cleaned")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

clean_feat_path = SAVE_DIR / "employee_features_clean.csv"
clean_full_path = SAVE_DIR / "employee_full_clean.csv"
provenance_path = SAVE_DIR / "cleaning_provenance.json"

df_feat.to_csv(clean_feat_path, index=False)
df_full.to_csv(clean_full_path, index=False)
with open(provenance_path, "w") as f:
    json.dump(PROVENANCE, f, indent=2)

print("Saved:\n -", clean_feat_path, "\n -", clean_full_path, "\n -", provenance_path)
report(df_full, "FINAL (clean)")


Saved:
 - artifacts/cleaned/employee_features_clean.csv 
 - artifacts/cleaned/employee_full_clean.csv 
 - artifacts/cleaned/cleaning_provenance.json
--- FINAL (clean) ---
shape: (999895, 12)

.dtypes:
 job_id                          string[python]
company_id                      string[python]
job_role                        string[python]
education                       string[python]
major                           string[python]
industry                        string[python]
years_experience                       Float64
distance_from_cbd                      Float64
is_outlier_years_experience            boolean
is_outlier_distance_from_cbd           boolean
salary_in_thousands                    Float64
is_outlier_salary                      boolean
dtype: object

# missing by column:
 is_outlier_salary               225
salary_in_thousands             225
job_id                            0
company_id                        0
education                         0
job_role         

,job_id,company_id,job_role,education,major,industry,years_experience,distance_from_cbd,is_outlier_years_experience,is_outlier_distance_from_cbd,salary_in_thousands,is_outlier_salary
0,JOB1362684407687,COMP37,CFO,MASTER,MATH,HEALTH,10.0,83.0,False,False,130.0,False
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3.0,73.0,False,False,101.0,False
2,JOB1362684407697,COMP56,JANITOR,HIGH_SCHOOL,NONE,HEALTH,24.0,30.0,False,False,102.0,False
3,JOB1362684407698,COMP7,CEO,MASTER,PHYSICS,EDUCATION,7.0,79.0,False,False,144.0,False
4,JOB1362684407699,COMP4,JUNIOR,NONE,NONE,OIL,8.0,29.0,False,False,79.0,False


(999895, 12)

In [ ]:
#cell 10.1 — Model-ready slice (drop rows with missing target only)
from pathlib import Path

TARGET = "salary_in_thousands"
assert TARGET in df_full.columns, f"Missing target '{TARGET}' in df_full"

df_train = df_full[df_full[TARGET].notna()].copy()
dropped = len(df_full) - len(df_train)

print("Model-ready shape:", df_train.shape)
print("Dropped rows (missing target):", dropped)

# Persist a readable reference table (non-encoded) for auditing
ART_DIR = Path("./artifacts/model_ready")
ART_DIR.mkdir(parents=True, exist_ok=True)
ref_path = ART_DIR / "train_reference.parquet"
df_train.to_parquet(ref_path, index=False)
print("Saved non-encoded training reference ->", ref_path)


Model-ready shape: (999670, 12)
Dropped rows (missing target): 225
Saved non-encoded training reference -> artifacts/model_ready/train_reference.parquet


In [ ]:
#final Cleaning QA — single PASS/FAIL gate

import pandas as pd
import numpy as np

# columns that must be fully populated after cleaning (features only)
must_have_no_na = [
    "job_id","company_id","job_role","education","major","industry",
    "years_experience","distance_from_cbd",
    "is_outlier_years_experience","is_outlier_distance_from_cbd"
]
missing_features = df_full[must_have_no_na].isna().sum()
assert missing_features.sum() == 0, f"Feature NAs remain:\n{missing_features[missing_features>0]}"

# key & format invariants
assert len(df_full) == len(df_feat), "Left-join row loss detected"
assert df_full["job_id"].is_unique, "Duplicate job_id in df_full"
assert df_full["job_id"].str.fullmatch(r"JOB\d+").all(), "Invalid job_id format"
cid = pd.Series(df_full["company_id"], dtype="string[python]")
assert ((cid.str.fullmatch(r"COMP\d+") | cid.eq("COMP_UNKNOWN")).fillna(False)).all(), \
       "company_id must be COMP#### or COMP_UNKNOWN"

# numeric ranges per cleaning policy
assert df_full["years_experience"].between(0, 50, inclusive="both").all(), "years_experience outside [0,50]"
assert df_full["distance_from_cbd"].between(0, 150, inclusive="both").all(), "distance_from_cbd outside [0,150]"

# target-only missing allowed
na_target = int(df_full["salary_in_thousands"].isna().sum())
assert na_target == int(df_full["is_outlier_salary"].isna().sum()), "Mismatch in salary vs salary-flag NAs"

# dtype sanity 
exp_strings = ["job_id","company_id","job_role","education","major","industry"]
for c in exp_strings:
    assert str(df_full[c].dtype).startswith("string"), f"{c} not string dtype"
for c in ["years_experience","distance_from_cbd","salary_in_thousands"]:
    assert str(df_full[c].dtype) in ("Float64","float64"), f"{c} not numeric Float64"
for c in ["is_outlier_years_experience","is_outlier_distance_from_cbd","is_outlier_salary"]:
    assert str(df_full[c].dtype) == "boolean", f"{c} not boolean dtype"

print("✅ Final cleaning checks passed. Missing salary rows:", na_target)


✅ Final cleaning checks passed. Missing salary rows: 225
